In [33]:
import scipy.io
import scipy
import matplotlib.pyplot as plt
import numpy as np
import os

In [53]:
ids = '01'
kernel_dir = f'/work/project/MEG_GOD/GOD_dataset/sbj{ids}/kernel'
ind_kernel_filename = 'results_MN_MEG_KERNEL_block001.mat'
common_kernel_filename = 'tess_cortex_pial_low.mat'

ch_index_path = '/work/project/MEG_GOD/GOD_dataset/channel_index.mat'
ch_index_path2 = '/work/project/MEG_GOD/GOD_dataset/channel_ricoh_acc1.mat'

# Kernel

In [10]:
ind_kernel_path = os.path.join(kernel_dir, ind_kernel_filename)
common_kernel_path = os.path.join(kernel_dir, common_kernel_filename)
print(ind_kernel_path)
print(common_kernel_path)
ind_kernel = scipy.io.loadmat(kernel_path)
common_kernel = scipy.io.loadmat(common_kernel_path)

/work/project/MEG_GOD/GOD_dataset/sbj01/kernel/results_MN_MEG_KERNEL_block001.mat
/work/project/MEG_GOD/GOD_dataset/sbj01/kernel/tess_cortex_pial_low.mat


In [12]:
print(ind_kernel.keys())
print('\n')
print(common_kernel.keys())

dict_keys(['__header__', '__version__', '__globals__', 'ImagingKernel', 'ImageGridAmp', 'Std', 'Whitener', 'SourceDecompSa', 'SourceDecompVa', 'nComponents', 'Comment', 'Function', 'Time', 'DataFile', 'HeadModelFile', 'HeadModelType', 'ChannelFlag', 'GoodChannel', 'SurfaceFile', 'Atlas', 'GridLoc', 'GridOrient', 'GridAtlas', 'Options', 'ColormapType', 'DisplayUnits', 'ZScore', 'nAvg', 'Leff', 'DataWhitener', 'History'])


dict_keys(['__header__', '__version__', '__globals__', 'Comment', 'Vertices', 'Faces', 'VertConn', 'VertNormals', 'Curvature', 'SulciMap', 'Atlas', 'iAtlas', 'tess2mri_interp', 'Reg', 'History', 'tess2tess_interp'])


In [50]:
ind_mat = ind_kernel['ImagingKernel']
common_mat = common_kernel['tess2tess_interp']['Wmat'][0,0].toarray()# common_kernel['tess2tess_interp']

In [51]:
print('ind_mat, shape', ind_mat.shape)
print('common_mat, shape', common_mat.shape)

ind_mat, shape (15002, 160)
common_mat, shape (15002, 15002)


# Ch Index

In [55]:
ch_index1 = scipy.io.loadmat(ch_index_path)
ch_index2 = scipy.io.loadmat(ch_index_path2)

In [57]:
ch_index2.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Comment', 'MegRefCoef', 'TransfMeg', 'TransfMegLabels', 'TransfEeg', 'TransfEegLabels', 'HeadPoints', 'Channel', 'IntraElectrodes', 'History', 'SCS', 'Projector'])

In [59]:
ch_index2['Channel']

array([[(array(['LF11'], dtype='<U4'), array(['MIT KIT system gradiometer size = 15.50  mm base = 50.00  mm'],
              dtype='<U60'), array(['MEG'], dtype='<U3'), array([], shape=(0, 0), dtype=uint8), array([[0.13379011, 0.13318963, 0.13506263, 0.13566311, 0.18215302,
                0.18155255, 0.18342555, 0.18402602],
               [0.02280378, 0.02876159, 0.02408618, 0.01812836, 0.03341947,
                0.03937728, 0.03470187, 0.02874405],
               [0.08591674, 0.08099675, 0.07510651, 0.08002651, 0.09286911,
                0.08794911, 0.08205888, 0.08697888]]), array([[0.96725834, 0.96725834, 0.96725834, 0.96725834, 0.96725834,
                0.96725834, 0.96725834, 0.96725834],
               [0.21231378, 0.21231378, 0.21231378, 0.21231378, 0.21231378,
                0.21231378, 0.21231378, 0.21231378],
               [0.13904737, 0.13904737, 0.13904737, 0.13904737, 0.13904737,
                0.13904737, 0.13904737, 0.13904737]]), array([[ 0.25,  0.25,  0.25,  0

In [79]:
cnt = 0
target_roi_indices = []
roi_blocks = [1, 2, 3, 4, 5, 24, 25, 26, 27, 28]
for i in roi_blocks:
    id_ = str(i).zfill(2)
    roi_path = f'/work/project/MEG_GOD/hyang/BData/ROI/hcp44_{id_}.mat'
    roi = scipy.io.loadmat(roi_path)
    # print(roi.keys())
    print(roi['vertex_ds'].shape) # vertex_ds => down sampled
    cnt+= roi['vertex_ds'].shape[0]
    target_roi_indices.append(roi['vertex_ds'])
target_roi_indices = np.concatenate(target_roi_indices, axis=0) - 1 # 0始まりなので
print(target_roi_indices.shape)
print(cnt)

(47, 1)
(85, 1)
(22, 1)
(32, 1)
(31, 1)
(52, 1)
(84, 1)
(28, 1)
(30, 1)
(38, 1)
(449, 1)
449


# create Kernel 

In [80]:
print('common_mat shape: ', common_mat.shape)
print('ind_mat shape: ', ind_mat.shape)
whole_brain_kernel = common_mat @ ind_mat
print('whole_brain_kernel shape: ', whole_brain_kernel.shape)
target_region_kernel = whole_brain_kernel[target_roi_indices[:,0]]
print('target_region_kernel shape: ', target_region_kernel.shape)

common_mat shape:  (15002, 15002)
ind_mat shape:  (15002, 160)
whole_brain_kernel shape:  (15002, 160)
target_region_kernel shape:  (449, 160)


In [83]:
# more effective solution
target_region_kernel2 = common_mat[target_roi_indices[:,0]] @ ind_mat
print(np.all(target_region_kernel == target_region_kernel2))

True
